# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
nhl_df['team'] = nhl_df['team'].str.replace(r'*', '')
nhl_df=nhl_df[nhl_df['year']==2018].drop([0,9,18,26])
nhl_df['W'] = pd.to_numeric(nhl_df['W'])
nhl_df['L'] = pd.to_numeric(nhl_df['L'])
nhl_df['win/loss ratio'] = nhl_df['W']/(nhl_df['W']+nhl_df['L'])
    
new = nhl_df["team"].str.rsplit(" ", expand = True)
new[3] = ''
for i in new.index:
        if (i in [3, 5, 13, 27]):
            new[3][i] = new[1][i] + ' ' + new[2][i]
        elif (i in [1, 14, 16, 17, 23, 29, 30]):
            new[3][i] = new[2][i]
        else:
            new[3][i] = new[1][i]
nhl_df['NHL'] = new[3]
    
RID = nhl_df['win/loss ratio'][[14,16,17]]
nhl_df['win/loss ratio'][[14,16,17]] = RID.mean()
KD = nhl_df['win/loss ratio'][[28,30]]
nhl_df['win/loss ratio'][[28,30]] = KD.mean()
#print(nhl_df[['NHL','win/loss ratio']])
    
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NHL'] = cities['NHL'].str.replace(r'\[.*\]|—', '')
cities['NHL'] = cities['NHL'].str.replace('IslandersDevils|Ducks', '')
cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
#print(cities[['Metropolitan area','Population (2016 est.)[8]','NHL']])

nhl_cities_df = nhl_df.merge(cities, how='inner', on='NHL')
#print(nhl_cities_df[['Metropolitan area','Population (2016 est.)[8]','NHL','win/loss ratio']])    
#print(nhl_cities_df['Population (2016 est.)[8]'])
#print(nhl_cities_df['win/loss ratio']) 
    

def nhl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
        
    population_by_region = nhl_cities_df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = nhl_cities_df['win/loss ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    (corr,p) = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return corr
nhl_correlation()

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
nba_df['team'] = nba_df['team'].str.replace(r'*', '')
nba_df['team'] = nba_df['team'].str.replace(r'\s\(.*\)', '')
nba_df=nba_df[nba_df['year']==2018]

nba_df['W'] = pd.to_numeric(nba_df['W'])
nba_df['L'] = pd.to_numeric(nba_df['L'])
nba_df['win/loss ratio'] = nba_df['W']/(nba_df['W']+nba_df['L'])

new = nba_df["team"].str.rsplit(" ", expand = True)
new[3] = ''
for i in new.index:
    if (i in [17]):
        new[3][i] = new[1][i] + ' ' + new[2][i]
    elif (i in [10,16,18,20,21,24,25]):
        new[3][i] = new[2][i]
    else:
        new[3][i] = new[1][i]
nba_df['NBA'] = new[3]
KN = nba_df['win/loss ratio'][[10,11]]
nba_df['win/loss ratio'][[10,11]] = KN.mean()
LC = nba_df['win/loss ratio'][[24,25]]
nba_df['win/loss ratio'][[24,25]] = LC.mean()
#print(nba_df[['NBA','win/loss ratio']])

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NBA'] = cities['NBA'].str.replace(r'\[.*\]|—', '')
cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
cities['NBA'] = cities['NBA'].str.replace('Nets|Clippers', '')
#print(cities[['Metropolitan area','Population (2016 est.)[8]','NBA']])

nba_cities_df = nba_df.merge(cities, how='inner', on='NBA')

def nba_correlation():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = nba_cities_df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = nba_cities_df['win/loss ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    
    (corr,p) = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr
nba_correlation()

-0.1765716025284462

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
mlb_df=mlb_df[mlb_df['year']==2018]
mlb_df['W'] = pd.to_numeric(mlb_df['W'])
mlb_df['L'] = pd.to_numeric(mlb_df['L'])
mlb_df['win/loss ratio'] = mlb_df['W']/(mlb_df['W']+mlb_df['L'])

new = mlb_df["team"].str.rsplit(" ", expand = True)
new[3] = ''
for i in new.index:
    if (i in [0,3,8]):
        new[3][i] = new[1][i] + ' ' + new[2][i]
    elif (i in [1,2,9,13,18,22,25,28,29]):
        new[3][i] = new[2][i]
    else:
        new[3][i] = new[1][i]
mlb_df['MLB'] = new[3]
YM = mlb_df['win/loss ratio'][[1,18]]
mlb_df['win/loss ratio'][[1,18]] = YM.mean()
DA = mlb_df['win/loss ratio'][[13,25]]
mlb_df['win/loss ratio'][[13,25]] = DA.mean()
GA = mlb_df['win/loss ratio'][[11,28]]
mlb_df['win/loss ratio'][[11,28]] = GA.mean()
CW = mlb_df['win/loss ratio'][[8,21]]
mlb_df['win/loss ratio'][[8,21]] = CW.mean()
#print(mlb_df[['MLB','win/loss ratio']])

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['MLB'] = cities['MLB'].str.replace(r'\[.*\]|—', '')
cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
cities['MLB'] = cities['MLB'].str.replace('Mets|Angels|Athletics|White Sox', '')
#print(cities[['Metropolitan area','Population (2016 est.)[8]','MLB']])

mlb_cities_df = mlb_df.merge(cities, how='inner', on='MLB')

def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = mlb_cities_df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = mlb_cities_df['win/loss ratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    
    (corr,p) = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr
mlb_correlation()

0.1502769830266931

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
nfl_df=nfl_df[nfl_df['year']==2018].drop([0,5,10,15,20,25,30,35])
nfl_df['team'] = nfl_df['team'].str.replace(r'[*+]', '')
nfl_df['W'] = pd.to_numeric(nfl_df['W'])
nfl_df['L'] = pd.to_numeric(nfl_df['L'])
nfl_df['win/loss ratio'] = nfl_df['W']/(nfl_df['W']+nfl_df['L'])

new = nfl_df["team"].str.rsplit(" ", expand = True)
new[3] = ''
for i in new.index:
    if (i in new[new[2].notna()].index):
        new[3][i] = new[2][i]
    else:
        new[3][i] = new[1][i]
nfl_df['NFL'] = new[3]
GJ = nfl_df['win/loss ratio'][[4,24]]
nfl_df['win/loss ratio'][[4,24]] = GJ.mean()
RC = nfl_df['win/loss ratio'][[17,36]]
nfl_df['win/loss ratio'][[17,36]] = RC.mean()
FR = nfl_df['win/loss ratio'][[19,38]]
nfl_df['win/loss ratio'][[19,38]] = FR.mean()
#print(nfl_df[['NFL','win/loss ratio']])

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NFL'] = cities['NFL'].str.replace(r'\[.*\]|—', '')
cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
cities['NFL'] = cities['NFL'].str.replace('Jets|Chargers|Raiders', '')
#print(cities[['Metropolitan area','Population (2016 est.)[8]','NFL']])

nfl_cities_df = nfl_df.merge(cities, how='inner', on='NFL')

def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = nfl_cities_df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = nfl_cities_df['win/loss ratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    (corr,p) = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr
nfl_correlation()

0.00492211214934943

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

#mlb_df=pd.read_csv("assets/mlb.csv")
mlb_cities_df['MLB'] = mlb_cities_df['win/loss ratio']
#print(mlb_cities_df[['Metropolitan area','MLB']])
#nhl_df=pd.read_csv("assets/nhl.csv")
nhl_cities_df['NHL'] = nhl_cities_df['win/loss ratio']
#print(nhl_cities_df[['Metropolitan area','NHL']])
#nba_df=pd.read_csv("assets/nba.csv")
nba_cities_df['NBA'] = nba_cities_df['win/loss ratio']
#print(nba_cities_df[['Metropolitan area','NBA']])
#nfl_df=pd.read_csv("assets/nfl.csv")
nfl_cities_df['NFL'] = nfl_cities_df['win/loss ratio']
#print(nfl_cities_df[['Metropolitan area','NFL']])
#cities=pd.read_html("assets/wikipedia_data.html")[1]
#cities=cities.iloc[:-1,[0,3,5,6,7,8]]

Big_Four_1 = mlb_cities_df[['Metropolitan area','MLB']].merge(nhl_cities_df[['Metropolitan area','NHL']], how='outer', on='Metropolitan area')
Big_Four_2 = nba_cities_df[['Metropolitan area','NBA']].merge(nfl_cities_df[['Metropolitan area','NFL']], how='outer', on='Metropolitan area')
Big_Four = Big_Four_1.merge(Big_Four_2, how='outer', on='Metropolitan area')
Big_Four['is_nan'] = 0
for i in Big_Four.index:
    Big_Four['is_nan'][i] = Big_Four.iloc[[i]].isna().sum().sum()
Big_Four = Big_Four.drop(Big_Four[Big_Four.is_nan == 3].index).drop(columns='is_nan')
#print(Big_Four)
#print(p)

def sports_team_performance():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    for i in sports:
        for j in sports:
            if i != j:
                (corr,p) = stats.ttest_rel(Big_Four[i], Big_Four[j],nan_policy='omit')
                p_values[i][j] = p
            
    #print(p_values)
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NFL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
